In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import math
from sklearn import svm, metrics
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.datasets._samples_generator import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, r2_score, accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

np.random.seed(0)

import seaborn as sns

sns.set(font_scale=1.2)

In [ ]:
#variables to use in this kaggle
(x,y,z) = 50,168.6,"learning"
print(x,y,z)
A = "indiaindiafkdjsgfdgdfindia"
print(A)
print(A*7,'\n')
print(A.replace('india',' REPLACED '))


In [ ]:
print('We want to show an example of Support Vector Machines for machine learning / data mining')
import pickle

recs = pd.read_csv("/kaggle/input/recipes-muffins-cupcakes-in-svm/recipes_muffins_cupcakes.csv")
print(recs.head())

In [ ]:
print('We want to plot our data using SNS')
sns.lmplot(data=recs, x='Flour', y='Sugar', hue='Type', palette='Set1', fit_reg=False, scatter_kws={"s": 70})
plt.show()

In [ ]:
print('the above plot shows some data as how it appears, though we want it to be more processed as to have it appear more precise')
type_label = np.where(recs['Type']=='Muffin',0,1)
rec_features = recs.columns.values[1:].tolist()
rec_features
ingredients = recs[['Flour','Sugar']].values
print(ingredients)

In [ ]:
print('now to get the model to fit to the pruned data')
model = svm.SVC(kernel='linear')
model.fit(ingredients,type_label)

In [ ]:
print('getting a hyperplane now')
w = model.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(30,60)
yy = a*xx - (model.intercept_[0] / w[1])


b = model.support_vectors_[0]
yy_down = a*xx+(b[1]-a*b[0])
b = model.support_vectors_[-1]
yy_up = a*xx+(b[1]-a*b[0])

print('this shows how the data can deliniate where the datapoints approach the plane')
sns.lmplot(data=recs, x='Flour', y='Sugar', hue='Type', palette='Set1', fit_reg=False, scatter_kws={"s": 70})
plt.plot(xx,yy,linewidth=2,color='black')
plt.plot(xx,yy_down,'k--')
plt.plot(xx,yy_up,'k--')
plt.plot(50,20,'yo',markersize='9')

In [ ]:
print('lets create a predictive fucntion using flower and sugar')
def muff_or_cup(flour,sugar):
    if(model.predict([[flour,sugar]]))==0:
        print('Youre looking at some Muffins')
    else:
        print('Youre looking at some cupcakes!')

print('lets use some values 50 flour and 20 sugar, what will we get?',muff_or_cup(50,20) )
#current time in video is around 1:19:55

In [ ]:
# Read the dataset
cars = pd.read_csv("/kaggle/input/cars-for-kmeans/cars (dataset for k means).csv")

# Handle missing values
#this method will not work as it just blanket changes the items that are strings even though they are numbers, it errors out
#x = cars.fillna(cars.mean())

#better working version
x = cars[cars.columns[:-1]]
#x = x.convert_objects(convert_numeric=True) , this is deperciated now, use the following
x= x._convert(numeric=True)
x.head()

#elm null values
for i in x.columns:
    x[i] = x[i].fillna(int(x[i].mean()))
for i in x.columns:
    print(x[i].isnull().sum())

# Perform K-means clustering and find the optimal number of clusters
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method
plt.plot(range(1, 11), wcss, marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()


kmeansA = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans = kmeansA.fit_predict(x)
x= x.values
print(y_kmeans,'\n',x)

In [ ]:
plt.scatter(x[y_kmeans == 0,0], x[y_kmeans == 0,1],s=100,c='red',label='Toyota')
plt.scatter(x[y_kmeans == 1,0], x[y_kmeans == 1,1],s=100,c='blue',label='Nissan')
plt.scatter(x[y_kmeans == 2,0], x[y_kmeans == 2,1],s=100,c='green',label='Honda')
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],s=300,c='yellow',label='Centroids')
plt.title('Clusters of car make')
plt.legend()
plt.show()

In [ ]:
#now we are going to user logestic regression to find some data on weather a tumor is malignent
tumor = pd.read_csv('/kaggle/input/tumor-size/data (1) (dataset for logistic).csv')
sns.jointplot(x='radius_mean',y='texture_mean',data = tumor)


In [ ]:
sns.heatmap(tumor.corr()) #plotting an unalterrted heatmap along with null values if any in the data
tumor.isnull().sum()

In [ ]:
#we will want to create an array with the worst possible data points and the diagnosis
worstX=tumor[['radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concave points_worst','symmetry_worst','fractal_dimension_worst'
]]
worstY=tumor['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(worstX, worstY, test_size=0.3, random_state=101)

#running the model without a max iteration limit will cause it to error out hence the 1000
logModel = LogisticRegression(max_iter=1000)
logModel.fit(X_train, y_train)

In [ ]:
#now we do the predictions
y_pred = logModel.predict(X_test)
print('the printer array shows how using a model it will classify if the tumor is benign or malignent')
print(y_pred)
print('\nNow we will see how well the results match up\n',classification_report(y_test,y_pred))

In [ ]:
#some linear algebra refreshers

matA = np.array([10,15])
matB = np.array([20,9])
matD = np.dot(matA,matB) #this is the dot product in linear algebra
print(matA,'\n',matB)
matC = matA.T+matB.T #using .T transposes the arrays
print(matC)
print((10*20)+(15*9))
print(matD)

In [ ]:
#lets do some calculus, specifically finding the limit of something 
#neural networks achive the speed that they need by first going establishing layer, then differenting the diffrent layers
# first layer will affect second layer and so on, this is proved by calculus
#Gradiant Descent

cur_x = 5 #algo starts at 3
rate = 0.1
precision = 0.5 # how close it should be to stopping
previous_step_size = 1
max_iters = 1000000 # one million iterations
iters = 0
df = lambda x: 2*(x+5) # gradiant of our function

while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #storing current value
    cur_x = cur_x - rate * df(prev_x) #evaluation the new value by the formula
    previous_step_size = abs(cur_x - prev_x) # the new x
    iters = iters +1 #iteration count
    print('Iteration',iters,'\nX value is',cur_x)
    
print('The local minimum occurs at', cur_x)
#scipi does this as a built in function buttt this can be more fun since you get to learn the diffrent functions


In [ ]:
dfD = pd.DataFrame({'Name': ['Jane','Michael','Willian','Rosy','Hana','Ferdie','Graeme'],'Salary':[50000,54000,50000,189000,55000,40000,59000]})
print('Means',dfD['Salary'].mean())
print('Median', dfD['Salary'].median())
print('Mode',dfD['Salary'].mode())

%matplotlib inline
salary = dfD['Salary']
salary.plot.hist(title='Salary Distribution', color='Grey',bins=25)
plt.axvline(salary.mean(),color='violet',linestyle='dashed',linewidth=2)
plt.axvline(salary.median(), color='red',linestyle='dashed',linewidth=2)
plt.show()

In [ ]:
#lets create a set to show some probabilities
set1 = {4,7}
print(set1)

#now we convert that set into a list to change it around
newList = [1,2,3,4, 4]
setFromList = set(newList)
print(setFromList)

from itertools import product as prod

ap = set([x for x in range(1,7)])
bp = set([x for x in range(1,7)])
pp = list(prod(ap,bp))

print('Ap is the set of all possible outcomes of a dice: ',ap)
print('Bp is the set of all possible outcomes of a dice: ',bp)
print('products ap and bp are true for all possible combinations of Ap and Bp thrown together:\n',pp)


#we can toss two dice and see what happens here

n_dice = 2
dice_faces = {1,2,3,4,5,6}
#now make the event space

event_space = set(prod(dice_faces, repeat=n_dice))
event_space

print('this is a new line')
for outcome in event_space:
    print(outcome,end =', ')
print()
print(len(event_space))

#now lets do it for a sum of something
faveOutcome = []
for outcome in event_space:
    x,y = outcome
    if (x+y)%3 == 0:
        faveOutcome.append(outcome)
faveOutcome = set(faveOutcome)

for f_outcome in faveOutcome:
    print(f_outcome, end=', ')
print()
print(len(faveOutcome))

prob = len(faveOutcome)/len(event_space)
print('The probability of getting a sum which is a multiple of 3 is: ',prob)



In [ ]:
#now for a sum of a multiple of 5 but not of 3
n_diceA = 5
dice_faces = {1,2,3,4,5,6}
#now make the event space

event_spaceA = set(prod(dice_faces, repeat=n_diceA))
faveOutcomes = []
for outcomesA in event_spaceA:
    d1,d2,d3,d4,d5 = outcomesA
    if (d1+d2+d3+d4+d5)%5 == 0 and (d1+d2+d3+d4+d5)%3!=0:
        faveOutcomes.append(outcomesA)
faveOutcomes = set(faveOutcomes)
len(faveOutcomes)
probA = len(faveOutcomes)/len(event_spaceA)
print('The probability of getting a sum which is a multiple of 5 and not 3 is: ',probA)

In [ ]:
#it is naive bayes time to learn and do
sna = pd.read_csv('/kaggle/input/socialnetworkads/SocialNetworkAds.csv')
print(sna.head())

xsna = sna.iloc[:,[2,3]].values
ysna = sna.iloc[:,4].values

X_train, X_test, y_train, y_test = train_test_split(xsna,ysna,test_size = 0.25, random_state = 0)

In [ ]:
#time to feature scale
scx = StandardScaler()
X_train = scx.fit_transform(X_train)
X_test = scx.transform(X_test)

print(X_train[:3])

In [ ]:
#now we fit the Naive Bayes to the training set, lets see what outliers there are
classifier = GaussianNB()
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)
print('This should show how much is correct\ntr is true pos, tl is false pos, br is false neg and bl is true neg')
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
#we want to graph this as to show it better
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:,0].min() - 1, stop = X_set[:,0].max() +1, step = 0.01),
                    np.arange(start = X_set[:,1].min() - 1, stop = X_set[:,1].max() +1, step = 0.01))
plt.contourf(X1,X2, classifier.predict(np.array([X1.ravel(),X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red','green')))
plt.xlim(X1.min(),X1.max())
plt.ylim(X2.min(),X2.max())

for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j,0], X_set[y_set == j,1],
               c=ListedColormap(('red','green'))(i), label = j)

plt.title('Naive Bayes (Training Set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

In [ ]:
#multiple linear regressions
%matplotlib inline
companies =pd.read_csv("/kaggle/input/1000companies/1000_Companies.csv")
xxx = companies.iloc[:, :-1].values #take everyrow except last one
yyy = companies.iloc[:, :4].values #take every column except last 4

companies.head()
sns.heatmap(companies.corr())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

companies =pd.read_csv("/kaggle/input/1000companies/1000_Companies.csv")
xxx = companies.iloc[:, :-1].values #take everyrow except last one
yyy = companies.iloc[:, -1].values #take every column except last 4

labelencoder = LabelEncoder()
xxx[:,3] = labelencoder.fit_transform(xxx[:,3])

# Define the indices of categorical columns
categorical_features_indices = [3]

# Create a ColumnTransformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features_indices)
    ],
    remainder='passthrough'  # You can specify other transformers for non-categorical columns
)

# Fit and transform your data
xxx = preprocessor.fit_transform(xxx)
#now we will remove the index column since its just false data
xxx=xxx[:,1:]
#its time to train it!
X_train, X_test, y_train,y_test = train_test_split(xxx, yyy, test_size =0.2, random_state = 0)
regressor = LinearRegression()
#I have an error here where the table data cant be converted from a string to numerical, the onehotencoder should be rectifiying this
#but currently it does not, need to investigate
# Train the model
regressor.fit(X_train, y_train)

#previously we had included y_train but this gave off some simaler results however beacuse it was a 2d array
#it could not be used for computing the R2 score which is only a 1d array algo
y_pred = regressor.predict(X_test)

# this fixes it yyy = companies.iloc[:, -1].values, taking out the : solved the issue
#companies.head()
print(y_pred)
print("Time to print out the coefficients for y=mx+b:",regressor.coef_)
print("This will calculate the intecept for y=mx+b:",regressor.intercept_)

# Obtain predictions using the trained model
# Evaluate the model, this will show how close/accurte we are from predicited trained model to actual
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)
#a score of 91% is pretty decent confidence though these numbers could be a bit higher, to do that youll need to 
#include more data, more repetitions of the model to make it worth while


In [ ]:
#image recognition time! this uses a built in sklearn dataset for numbers
digits = load_digits()

print("Image data shape", digits.data.shape)
print("Label data shape", digits.target.shape)

In [ ]:
#what this will do is show the plots of training data of numbers that will show the shapes
plt.figure(figsize=(20,4))
for index, (image, label) in enumerate(zip(digits.data[0:5], digits.target[0:5])):
    plt.subplot(1,5,index+1)
    plt.imshow(np.reshape(image,(8,8)),cmap=plt.cm.gray)
    plt.title('training: %i\n' % label, fontsize = 20)

In [ ]:
#the data will now be trained to show how good it looks
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.23, random_state=2)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#implementiing the data that was trained from the digits dataset
#this comes back with a max iterations so will have to be redone to boost the iteration sup to 1000 maybe?

logisticRegr = LogisticRegression(max_iter=2000)
logisticRegr.fit(x_train,y_train)
print(logisticRegr.predict(x_test[1].reshape(1,-1)))

logisticRegr.predict(x_test[0:10])
predictNum = logisticRegr.predict(x_test)

scores = logisticRegr.score(x_test,y_test)
print("Accuracy:",scores)


In [ ]:
#this is to show the confusion matrix based on how accurate the model trained
cm = metrics.confusion_matrix(y_test,predictNum)
print("Confusion Matrix for number prediction\n",cm)

In [ ]:
print('New plot')
plt.figure(figsize=(9,9))
sns.heatmap(cm,annot=True,fmt=".3f",linewidth=.5,square=True,cmap='Blues_r');
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
all_sample_title = 'Accuracy Score: {0}'.format(scores)
plt.title(all_sample_title, size= 15)

In [ ]:
print('now another graph, this will show predicted vs actual')
index = 0
classifiedIndex = []
for predict, actual in zip(predictNum, y_test):
    if predict == actual:
        classifiedIndex.append(index)
    index += 1

plt.figure(figsize=(20,3))
for plotIndex, wrong in enumerate(classifiedIndex[0:4]):
    plt.subplot(1,4,plotIndex +1)
    plt.imshow(np.reshape(x_test[wrong],(8,8)), cmap=plt.cm.gray)
    plt.title("predicted: {}, Actual: {}".format(predictNum[wrong],y_test[wrong]),fontsize=20)

In [ ]:
#this will be a mainly confusion matrix focused demo
heart =pd.read_csv("/kaggle/input/heart-disease-cleveland-uci/heart_cleveland_upload.csv")
print('column 13 has condition instead of target, also the data seems to be a little big off')
heart


In [ ]:
#we are dropping the last column as the data is not needed
#heart = data, yH & xH are to diffrenetiate from the above variables
yH = heart['condition']
xH = heart.drop('condition',axis = 1)
print(xH.head())

#this will use a standard training model for preprocessing, nothing fancy, 20% test size 80% train
X_train, X_test, y_train, y_test, = train_test_split(xH,yH, test_size = 0.2, random_state = 42)

scalerH = StandardScaler()
scaleH = scalerH.fit(X_train)
X_train = scaleH.transform(X_train)
X_test = scaleH.transform(X_test)

#now creating a standard logistic regression
modelH = LogisticRegression()
modelH.fit(X_train,y_train)
predH = modelH.predict(X_test)

In [ ]:
#this is to get the accuracy score only for our model
scoreH = accuracy_score(y_test,predH)
print('This is how accurate our model is based on our trained data from the UCI heart CSV: ',scoreH)

#now well need the confusion matrix to get all the good stuff
print('Now viewing this data as such can be a little confusing, thankfully we can pull it apart just below')
confusion_matrix(y_test,predH)



In [ ]:
tn,fp,fn,tp = confusion_matrix(y_test,predH).ravel()
print('Here is our True Negative: ',tn,'\nHere is our False Negative: ',fn,'\nHere is our True Positive: ',tp,'\nHere is our False Positive: ',fp)

In [ ]:
#Welcome one and all to a python decsion tree that will see if a car loan will default!

balance_data = pd.read_csv('/kaggle/input/car-loan-decision-tree-repayment/CarLoanRepaymentDecisionTree.csv',sep = ',', header = 0)
balance_data = balance_data.drop('Unnamed: 5', axis=1)
balance_data.head() 

In [ ]:
#we want to find out the length of the CSV file since it contains over 1000 records
print('Dataset Length:: ',len(balance_data))
print('Dataset Shape:: ',balance_data.shape)

In [ ]:
#to figure out how to train the Decision tree, the data itself has to be sifted to figure out what will give out the max gain
xCarLoan = balance_data.values[:,1:5]
yCarLoan = balance_data.values[:,0]

#splitting our dataset up as one does to train and test it against it self
x_train, x_test, y_train, y_test = train_test_split(xCarLoan,yCarLoan, test_size=0.3, random_state=100)

#this function will find the entropy of a system based on how the data can be shuffled 
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100, max_depth = 3, min_samples_leaf = 5)
clf_entropy.fit(x_train, y_train)


In [ ]:
#We will want to predict something with this data now
y_pred_en = clf_entropy.predict(x_test)
y_pred_en

In [ ]:
#now we will see the accuracy, it is somewhat on point but could be better
print('Accuracy of this Car Loan model is : ',accuracy_score(y_test,y_pred_en)*100)

In [ ]:
#Delving into a Random Forest now

irisA = load_iris()
irisB = pd.read_csv('/kaggle/input/iris-random-forest/iris.csv')

#for iA we are following the built in data set
#for iB we are using a downloaded version, this will help compare the data between the two
iA = pd.DataFrame(irisA.data,columns=irisA.feature_names)
iB = pd.DataFrame(irisB)
iB = iB.drop('Id', axis=1)


iA.head()

In [ ]:
iB.head()

In [ ]:
#it appears we are missing a column in the built in database iris whereas the downloaded copy of iris already shows it
#lets add the species so we know whats happeneing

iA['Species'] = pd.Categorical.from_codes(irisA.target,irisA.target_names)
iA.head()

In [ ]:
#here we do a train/test with the built in database
iA['is_train'] = np.random.uniform(0,1,len(iA)) <= .75
iA.head()

In [ ]:
#Now this is with the downloaded database, take a look at the is_train column
iB['is_train'] = np.random.uniform(0,1,len(iB)) <= .75
iB.head()

In [ ]:
#Lets create new dataframes for our trained data
trainA, testA = iA[iA['is_train']==True], iA[iA['is_train']==False]
trainB, testB = iB[iB['is_train']==True], iB[iB['is_train']==False]

print('The number of observations in the built in training data:',len(trainA),'\nAs well as the number of observations in the test data:',len(testA))
print('\n')
print('This is the trained data from a downloaded copy of Iris, the training data:', len(trainB),'\nHere is the tested data:',len(testB))


def trainPercent(trainA, trainB):
    max_value = max(len(trainA), len(trainB))
    similarity = 1 - abs(len(trainA) - len(trainB)) / max_value
    percentA = similarity * 100

    return percentA

# Similarly, modify the testPercent function
def testPercent(testA, testB):
    max_value = max(len(trainA), len(trainB))
    similarity = 1 - abs(len(trainA) - len(trainB)) / max_value
    percentB = similarity * 100

    return percentB


print('\nNow beacuse we want to calculate to see the similarity of the two databases\nWe ran the numbers thru a small function to calculate that')
print('The train data percentage: ',trainPercent(trainA,trainB),'\nThe test data percentage:  ',testPercent(testA, testB))
print('In the end, both datasets are pretty simaler though right now they arent really quantified wihtout units')

featuresA = iA.columns[:4]
featuresB = iB.columns[:4]

#converting the genus/species into digits for easier quantifying. Allows the computer to understand it
yA = pd.factorize(trainA['Species'])[0]
yB = pd.factorize(trainB['Species'])[0]

print(yA)
print(yB)

#random forest classifer
#after cleaning up the data and getting it ready, now we can implement the classifer for a good prediction

clfA = RandomForestClassifier(n_jobs=2, random_state=0)
clfB = RandomForestClassifier(n_jobs=2, random_state=0)

clfA.fit(trainA[featuresA],yA)
clfB.fit(trainB[featuresB],yB)




In [ ]:
#implementing the random forest
print('Built in database',clfA.predict(testA[featuresA]),'\n')
print('Downloaded File',clfB.predict(testB[featuresB]),'\n')

In [ ]:
print('Built in database\n',clfA.predict_proba(testA[featuresA])[0:10],'\n')
print('Downloaded database\n',clfB.predict_proba(testB[featuresB])[0:10],'\n')

In [ ]:
#lets map some plant names so that we know what could come up, this will be for the first 5 observations
predsA = irisA.target_names[clfA.predict(testA[featuresA])]
#predsB = irisB.target_names[clfB.predict(testB[featuresB])], there is target_names in the downloaded dataset
predsB = irisA.target_names[clfB.predict(testB[featuresB])]

print(predsA[0:25],'\n')
print(predsB[0:25],'\n')

print('Viewing Actual data, not test\n',testA['Species'].head(),'\n\nActual data from the download\n',testB['Species'].head())

#now lets combine actual and test data into one confusion matrix!
irisCMA = pd.crosstab(testA['Species'],predsA, rownames=['Actual Species'], colnames=['Predicted Species'])
irisCMB = pd.crosstab(testB['Species'],predsB, rownames=['Actual Species'], colnames=['Predicted Species'])
print('\nThis is the built in database for Iris\n',irisCMA,'\n\nThis is the downloaded database\n',irisCMB)



In [ ]:
#KNN model of diabetes

diabetes = pd.read_csv('/kaggle/input/knndiabetes/diabetes.csv')
#time to replace the zeros with averaged data, this can skew the outcome of the KNN

zero_not_accepted = ['Glucose','BloodPressure','SkinThickness','BMI','Insulin']
for column in zero_not_accepted:
    diabetes[column] = diabetes[column].replace(0, np.NaN)
    mean = int(diabetes[column].mean(skipna=True))
    diabetes[column] = diabetes[column].replace(np.NaN, mean)

#now lets split up the adjusted dataset to train our model

xD = diabetes.iloc[:, 0:8]
yD = diabetes.iloc[:, 8]
X_train, X_test, y_train, y_test = train_test_split(xD, yD, random_state=0, test_size=0.2)

#this is to expand the data so that it is uniform
sc_xD = StandardScaler()
X_train = sc_xD.fit_transform(X_train)
X_test = sc_xD.transform(X_test)

classifierD = KNeighborsClassifier(n_neighbors = 11, p=2, metric='euclidean')
classifierD.fit(X_train,y_train)

predictD = classifierD.predict(X_test)

cmD = confusion_matrix(y_test, predictD)
print('Firstly, the confusion matrix, down diagnoal towards the right indicates correctness\n',cmD)

print('\nHere is our F1 score: \n',f1_score(y_test,predictD))
print('\nHere is the accuracy of the model: \n',accuracy_score(y_test,predictD))
print('\nOverall the model with the limited data was somewhat realistic but it could use more to chrun through')

In [ ]:
#Support Vector Machines 
#another staple of Machine Learning programs, SVM's can help to transform data thats usually massive, needs to have its bias cut or simply want a stable algo

#this is a sample run of SVM using the built in blob database in order to sort them out and see whos the blobbiest

#plain ol variables, we want 800 intakes and 15 types of blobs
xBlob, yBlob = make_blobs(n_samples = 80, centers = 7, random_state = 20)
#having the SVM algo create from a 1D mixxed bag to a 2D graph
clfBlob = svm.SVC(kernel = 'linear', C=1000)
clfBlob.fit(xBlob,yBlob)

plt.scatter(xBlob[:,0], xBlob[:,1], c=yBlob, s=30, cmap=plt.cm.Paired)

In [ ]:
#now that weve established and ran the SVM on the blob data, we will need to do some new additional data to be predicted
newBlobData = [[3,4],[5,6]]
print(clfBlob.predict(newBlobData))
#more or less shows that its accurate to a degree but its just blobs so lets make something more useful

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets._samples_generator import make_blobs
clfNew = svm.SVC(kernel = 'linear', C = 1000)
clfNew.fit(xBlob,yBlob)
plt.scatter(xBlob[:,0],xBlob[:,1], c=yBlob, s=30, cmap=plt.cm.Paired)
plt.show



In [ ]:
# the memoryy hog code
#ill have to run this locally as kaggle runs out of memory for the free version at least, 30gbs+ of ram!
ax = plt.gca()
xlimB = ax.get_xlim()
ylimB = ax.get_ylim()

xxB = np.linspace(xlimB[0],xlimB[1],30000)
yyB = np.linspace(ylimB[0], ylimB[1],30000)
xxB, yyB = np.meshgrid(yyB,xxB)
xyB = np.vstack([xxB.ravel(),yyB.ravel()]).T
zB = clfNew.decision_function(xyB).reshape(xxB.shape)

ax.contour (xxB, yyB, zB, colors = 'k', levels=[-1,0,1],
           alpha = 0.5,
           linestyles=['--','-','--'])

ax.scatter(clfNew.support_vectors_[:,0],
          clfNew.support_vectors_[:,1], s=100,
          linewidth=1, facecolors='none')
plt.show()

In [ ]:
#now applying a multiple linear regression model
"""Taking the data from California Housing, we train 75% and test the rest to get a good idea on what the dataset is
There is an error check done so as to verify that the data aint to far off skewed, it comes back at 0.506 which is a little over half wrong?
I suppose thats fine depending on your standards

The rest of the code then just spurts out some coefficent percentages"""

lreg = LinearRegression()
lreg.fit(x_train,y_train)

lreg_y_pred = lreg.predict(x_test)

mean_squared_error = np.mean((lreg_y_pred - y_test)**2)
print('Mean squared error on test set: ', mean_squared_error)

lreg_coefficient = pd.DataFrame()
lreg_coefficient["Columns"] = x_train.columns
lreg_coefficient['Coefficient estimate'] = pd.Series(lreg.coef_)
print(lreg_coefficient)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
boston_dataset = fetch_california_housing()

#had to use a differnt data set since boston is no longer in the sklearn framework
#this was mostly due to some racial data, which I suppose is fine but what if a person wants to use it since it has that?

boston_pd = pd.DataFrame(boston_dataset.data)
boston_pd.columns = boston_dataset.feature_names
boston_pd_target = np.asarray(boston_dataset.target)
boston_pd['House Price'] = pd.Series(boston_pd_target)

XBos = boston_pd.iloc[:, :-1]
YBos = boston_pd.iloc[:, -1]

print(boston_pd.head())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(boston_pd.iloc[:, :-1], boston_pd.iloc[:, -1], test_size = 0.25)
print('Train data shape of x = % s and Y = % s : '%(x_train.shape, y_train.shape))
print('Test data shape of X = % s and Y = % s : '%(x_test.shape, y_test.shape))

In [ ]:
#now to apply a multiple level linear regression model
#it will error check the test data, which makes it lean a little skewed & bias
#then it will print out all the coefficient data that we need to adjust

lreg = LinearRegression()
lreg.fit(x_train,y_train)

lreg_y_pred = lreg.predict(x_test)

mean_squared_error = np.mean((lreg_y_pred - y_test)**2)
print('Mean squared error on the test set: ',mean_squared_error)

lreg_coefficient = pd.DataFrame()
lreg_coefficient['Columns'] = x_train.columns
lreg_coefficient['Coefficent Estimate'] = pd.Series(lreg.coef_)
print(lreg_coefficient)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
color = ['tab:gray', 'tab:blue', 'tab:orange','tab:green','tab:red', 'tab:brown','tab:pink', 'tab:gray','tab:olive','tab:cyan','tab:orange','tab:green','tab:blue','tab:olive']

ax.bar(lreg_coefficient['Columns'], lreg_coefficient['Coefficent Estimate'], color = color)
ax.spines['bottom'].set_position('zero')

plt.style.use('ggplot')
plt.show()

In [ ]:
from sklearn.linear_model import Ridge
ridgeR = Ridge(alpha = 1)
ridgeR.fit(x_train,y_train)
y_pred = ridgeR.predict(x_test)

mean_squared_error_ridge = np.mean((y_pred - y_test)**2)
print('Mean squared error on test set with Ridge', mean_squared_error_ridge)

ridge_coefficient = pd.DataFrame()
ridge_coefficient['Columns'] = x_train.columns
ridge_coefficient['Coefficent Estimate'] = pd.Series(ridgeR.coef_)

print(ridge_coefficient)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
color = ['tab:gray', 'tab:blue', 'tab:orange','tab:green','tab:red', 'tab:brown','tab:pink', 'tab:gray','tab:olive','tab:cyan','tab:orange','tab:green','tab:blue','tab:olive']

ax.bar(ridge_coefficient['Columns'], ridge_coefficient['Coefficent Estimate'], color = color)
ax.spines['bottom'].set_position('zero')

plt.style.use('ggplot')
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 1)
lasso.fit(x_train,y_train)
y_pred1 = lasso.predict(x_test)

mean_squared_error_lasso = np.mean((y_pred1 - y_test)**2)
print('Mean squared error on test set with Lasso', mean_squared_error_lasso)

lasso_coefficient = pd.DataFrame()
lasso_coefficient['Columns'] = x_train.columns
lasso_coefficient['Coefficent Estimate'] = pd.Series(lasso.coef_)

print(lasso_coefficient)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
color = ['tab:gray', 'tab:blue', 'tab:orange','tab:green','tab:red', 'tab:brown','tab:pink', 'tab:gray','tab:olive','tab:cyan','tab:orange','tab:green','tab:blue','tab:olive']

ax.bar(ridge_coefficient['Columns'], lasso_coefficient['Coefficent Estimate'], color = color)
ax.spines['bottom'].set_position('zero')

plt.style.use('ggplot')
plt.show()

In [ ]:
#An example of Principal Component Analysis
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
cancer.keys()

dfc = pd.DataFrame(cancer['data'],columns = cancer['feature_names'])
dfc.head()

from sklearn.preprocessing import StandardScaler
scalerC = StandardScaler()
scalerC.fit(dfc)
scaled_data = scalerC.transform(dfc)

from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(scaled_data)

In [ ]:
#this took 30 features and smashed them down to two features
x_pca = pca.transform(scaled_data)
print(scaled_data.shape)
print(x_pca.shape)


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x_pca[:,0],x_pca[:,1],c=cancer['target'],cmap='plasma')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
#this graph shows two components from the previous 30, easy to show but hard to display all that info

In [ ]:
#we can show the individual components themselves and how often they occur though
df_comp = pd.DataFrame(pca.components_,columns=cancer["features_name"])
plt.figure(figsize=(12,6))
sns.heatmap(df_comp,cmap='plasma')

In [18]:
#final implmentation of covid prediction analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import math
import time
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

import datetime
import operator
plt.style.use('fivethirtyeight')
%matplotlib inline

#data was initally sorted in the JHU CSSE repossirtoy but beacuase of the way it was stored, it may have been too long a path
confirmed_cases = pd.read_csv('/kaggle/input/jhu-csse-covid19-confirmed/time_series_covid19_confirmed_global.csv')
deaths_reported = pd.read_csv('/kaggle/input/jhu-csse-covid19-deaths/time_series_covid19_deaths_global.csv')
recovered_cases = pd.read_csv('/kaggle/input/jhu-csse-covid19-recovered/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('/kaggle/input/jhu-csse-covid19-june19th2022report/06-19-2022.csv')

#editing columns to prepare the data, this will use the confiremd cases column as the main index for the other two
cols = confirmed_cases.keys()

#only allowing certain columns to come thru
confirmed = confirmed_cases.loc[:, cols[4]:cols[-1]]
deaths = deaths_reported.loc[:, cols[4]:cols[-1]]
recoveries = recovered_cases.loc[:, cols[4]:cols[-1]]



In [19]:
#huge dump of empty lists to showcase all the countries
dates = confirmed.keys()
world_cases = []
total_deaths = []
mortality_rate = []
recovery_rate = []
total_recovered = []
total_active = []

china_cases = []
italy_cases = []
us_cases = []
spain_cases = []
france_cases = []
germany_cases = []
uk_cases = []
russia_cases = []
india_cases = []

china_deaths = []
italy_deaths = []
us_deaths = []
spain_deaths = []
france_deaths = []
germany_deaths = []
uk_deaths = []
russia_deaths = []
india_deaths = []

china_recoveries = []
italy_recoveries = []
us_recoveries = []
spain_recoveries = []
france_recoveries = []
germany_recoveries = []
uk_recoveries = []
russia_recoveries = []
india_recoveries = []

In [22]:
#giant for loop to dump data inside

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)
    
    china_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='China'][i].sum())
    italy_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='Italy'][i].sum())
    us_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='US'][i].sum())
    spain_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='Spain'][i].sum())
    france_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='France'][i].sum())
    germany_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='Germany'][i].sum())
    uk_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='United Kingdom'][i].sum())
    russia_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='Russia'][i].sum())
    india_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='India'][i].sum())
    
    china_deaths.append(deaths_reported[deaths_reported['Country/Region']=='China'][i].sum())
    italy_deaths.append(deaths_reported[deaths_reported['Country/Region']=='Italy'][i].sum())
    us_deaths.append(deaths_reported[deaths_reported['Country/Region']=='US'][i].sum())
    spain_deaths.append(deaths_reported[deaths_reported['Country/Region']=='Spain'][i].sum())
    france_deaths.append(deaths_reported[deaths_reported['Country/Region']=='France'][i].sum())
    germany_deaths.append(deaths_reported[deaths_reported['Country/Region']=='Germany'][i].sum())
    uk_deaths.append(deaths_reported[deaths_reported['Country/Region']=='United Kingdom'][i].sum())
    russia_deaths.append(deaths_reported[deaths_reported['Country/Region']=='Russia'][i].sum())
    india_deaths.append(deaths_reported[deaths_reported['Country/Region']=='India'][i].sum())
    
    china_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='China'][i].sum())
    italy_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='Italy'][i].sum())
    us_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='US'][i].sum())
    spain_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='Spain'][i].sum())
    france_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='France'][i].sum())
    germany_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='Germany'][i].sum())
    uk_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='United Kingdom'][i].sum())
    russia_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='Russia'][i].sum())
    india_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='India'][i].sum())